In [1]:
### for emotion detection
https://github.com/monologg/GoEmotions-pytorch
### For toxicity detection
perspective api
### For politeness detection
https://github.com/AlafateABULIMITI/politeness-detection
### For language quality
https://github.com/WanzhengZhu/GPS/blob/master/language_quality.py
### Other metrics are aleady added add two metric from the GPS model.
### Implement GPS model as a baseline.
save_path='../HULK_new/Counterspeech/Saved_Models'

SyntaxError: invalid syntax (<ipython-input-1-92ae64a6dc1e>, line 2)

In [14]:
import json
import time
from tqdm.notebook import tqdm

In [2]:
#BLEU, METEOR, Diversity, Novelty
import glob
import json
from loguru import logger
import pandas as pd 
from Generation.eval import *
path_datasets = '/home/adarsh-binny' + '/HULK_new/Counterspeech/Datasets'
path_result   = '/home/adarsh-binny' + '/HULK_new/Counterspeech/Results'
save_path     = '/home/adarsh-binny' + '/HULK_new/Counterspeech/metrics_results/'

In [3]:
# Cell for Politeness evaluation

import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

from torch import cuda

def normalise_text(text):
    text = text.strip()
    text = text.lower() # lowercase
    text = text.replace(r"\#","") # replaces hashtags
    text = text.replace(r"http\S+","URL")  # remove URL addresses
    text = text.replace(r"@","")
    text = text.replace(r"[^A-Za-z0-9()!?\'\`\"]", " ")
    text = text.replace("\s{2,}", " ")
    text = text.replace(r"\#","")
    return text

VALID_BATCH_SIZE = 16

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")



In [4]:
torch.cuda.set_device(1)

In [5]:
device = "cuda" if cuda.is_available() else "cpu"

In [6]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = BertModel.from_pretrained("bert-base-uncased", return_dict=False)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 8)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = BERTClass()
model.to(device);


In [7]:
PATH = "/home/adarsh-binny/HULK_new/Counterspeech/Saved_Models/Metric_Models/Politeness/model_small.pt"
model.load_state_dict(torch.load(PATH));

In [8]:

class PDataset(Dataset):
    def __init__(self, hypo, tokenizer, max_len=200):
        self.len = len(hypo)
        self.data = hypo
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        text = self.data[index]
        #print(text)
        text = normalise_text(text)
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

    
def calc_politeness(hypo):
    N =len(hypo)
    print("Length of Hypothesis: ", N)
    hypo_ = PDataset(hypo, tokenizer, 200)
    testing_loader = DataLoader(hypo_, **test_params)
    model.eval()
    tr_loss = 0
    n_correct = 0 
    n_wrong = 0
    total = 0
    n1 = N//8
    if N%VALID_BATCH_SIZE!=0:
        n1+=1
    
    with torch.no_grad():
        with tqdm(n1) as pbar:
            for _, data in enumerate(testing_loader, 0):
                #print(data)
                ids = data['ids'].to(device, dtype = torch.long)
                mask = data['mask'].to(device, dtype = torch.long)
                outputs = model(ids, mask).squeeze()

                outputs = torch.nn.Softmax(dim=1)(outputs)
                
                #print(outputs.shape)
                #print(outputs[:,1])
                output = torch.argmax(outputs, dim=1)
                #print(output)
                total+= sum(output)
                pbar.update()
    return total/N
    

In [19]:
def metrics(trained_on,tested_on):
    train_path = glob.glob(path_datasets+'/*'+trained_on+'*/*rain*')[0]
    gen        = glob.glob(path_result+'/*'+trained_on+'*'+tested_on+'*1628*_updated')
    gen += glob.glob(path_result+'/*'+trained_on+'*'+tested_on+'*162*_*')
    ref        = glob.glob(path_result+'/*'+tested_on+'*'+'references'+'*')[0]
    
#     print(gen,ref)
    with open(ref, 'r') as file:
        ref_dict   = json.loads(file.read())
        
    scores = {}
    for files in gen:
        with open(files, 'r') as file:
            gen_dict  = json.loads(file.read())
        #emotion = gen_dict['params']['task_name'][0][1]
        #emotion = gen_dict['params']['task_name'][0][1]
        print(files)
        emotion = "politeness"
        print(gen_dict['params']['task_name'])
        print(gen_dict['params']['disc_weight'])
        if ("polite" in emotion or "gedi" not in files)==False:
            continue
        print(files)
        gpu_id    = gen_dict['params']['gpu_id']
        hypo = []
        refs = []
        signature = files.split("_")[-1].split(".")[0]
        for key in gen_dict['samples']:
            for sentences in gen_dict['samples'][key]['counterspeech_model']:
                hypo.append(sentences)
                refs.append(ref_dict['samples'][key]['counterspeech_model'])
        
        train = pd.read_csv(train_path)
        train_set = list(zip(train['initiator_message'].tolist(),train['reply_message'].tolist()))
        params = hypo
        average_score = calc_politeness(hypo)
        key = trained_on+'_'+tested_on+'_'+emotion+'_'+signature+'_'+str(gpu_id)
        scores[key] = average_score
        logger.info(f'Key:{key}--Politeness_Score :{average_score}')
    
    json.dump(scores, open(save_path +tested_on+'_politeness.json','w'),indent = 4)    

****Reddit****

In [20]:
metrics('Reddit','Reddit')

/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Reddit_DialoGPT-medium_on_Reddit_gedi_huggingface_1629484650_updated.json
[['Politeness', 'polite']]
[30]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Reddit_DialoGPT-medium_on_Reddit_gedi_huggingface_1629484650_updated.json
Length of Hypothesis:  100


2021-08-21 18:58:59.345 | INFO     | __main__:metrics:39 - Key:Reddit_Reddit_politeness_updated_0--Politeness_Score :6



/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Reddit_DialoGPT-medium_on_Reddit_gedi_huggingface_1629489580_single.json
[['Toxicity', 'toxic']]
[1]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Reddit_DialoGPT-medium_on_Reddit_gedi_huggingface_1629489580_single.json
Length of Hypothesis:  100


2021-08-21 18:59:00.382 | INFO     | __main__:metrics:39 - Key:Reddit_Reddit_politeness_single_0--Politeness_Score :4



/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Reddit_DialoGPT-medium_on_Reddit_gedi_huggingface_1629486310_updated.json
[['Politeness', 'polite']]
[1]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Reddit_DialoGPT-medium_on_Reddit_gedi_huggingface_1629486310_updated.json
Length of Hypothesis:  100


...


/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Reddit_DialoGPT-medium_on_Reddit_gedi_huggingface_1629550279_multi_2.json
[['Emotion', 'joy'], ['Politeness', 'polite']]
[1, 1]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Reddit_DialoGPT-medium_on_Reddit_gedi_huggingface_1629550279_multi_2.json
Length of Hypothesis:  100


2021-08-21 18:59:06.901 | INFO     | __main__:metrics:39 - Key:Reddit_Reddit_politeness_2_0--Politeness_Score :5



/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Reddit_DialoGPT-medium_on_Reddit_gedi_huggingface_1629488890_single.json
[['Emotion', 'joy']]
[1]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Reddit_DialoGPT-medium_on_Reddit_gedi_huggingface_1629488890_single.json
Length of Hypothesis:  100


2021-08-21 18:59:07.820 | INFO     | __main__:metrics:39 - Key:Reddit_Reddit_politeness_single_0--Politeness_Score :5


TypeError: Object of type Tensor is not JSON serializable

****CONAN****

In [21]:
metrics('CONAN','CONAN')

/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-CONAN_DialoGPT-medium_on_CONAN_gedi_huggingface_1629485151_updated.json
[['Politeness', 'polite']]
[30]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-CONAN_DialoGPT-medium_on_CONAN_gedi_huggingface_1629485151_updated.json
Length of Hypothesis:  86


2021-08-21 19:02:01.468 | INFO     | __main__:metrics:39 - Key:CONAN_CONAN_politeness_updated_0--Politeness_Score :5



/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-CONAN_DialoGPT-medium_on_CONAN_gedi_huggingface_1629486713_updated.json
[['Politeness', 'polite']]
[1]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-CONAN_DialoGPT-medium_on_CONAN_gedi_huggingface_1629486713_updated.json
Length of Hypothesis:  86


2021-08-21 19:02:02.074 | INFO     | __main__:metrics:39 - Key:CONAN_CONAN_politeness_updated_0--Politeness_Score :4



/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-CONAN_DialoGPT-medium_on_CONAN_gedi_huggingface_1629489279_single.json
[['Emotion', 'joy']]
[1]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-CONAN_DialoGPT-medium_on_CONAN_gedi_huggingface_1629489279_single.json
Length of Hypothesis:  86


...


/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-CONAN_DialoGPT-medium_on_CONAN_gedi_huggingface_1629485999_updated.json
[['Politeness', 'polite']]
[5]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-CONAN_DialoGPT-medium_on_CONAN_gedi_huggingface_1629485999_updated.json
Length of Hypothesis:  86


2021-08-21 19:02:05.282 | INFO     | __main__:metrics:39 - Key:CONAN_CONAN_politeness_updated_0--Politeness_Score :4



/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-CONAN_DialoGPT-medium_on_CONAN_gedi_huggingface_1629550796_multi_2.json
[['Emotion', 'joy'], ['Politeness', 'polite']]
[1, 1]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-CONAN_DialoGPT-medium_on_CONAN_gedi_huggingface_1629550796_multi_2.json
Length of Hypothesis:  86


2021-08-21 19:02:05.890 | INFO     | __main__:metrics:39 - Key:CONAN_CONAN_politeness_2_0--Politeness_Score :4


TypeError: Object of type Tensor is not JSON serializable

****Gab****

In [22]:
metrics('Gab','Gab')

/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Gab_DialoGPT-medium_on_Gab_gedi_huggingface_1629485810_updated.json
[['Politeness', 'polite']]
[5]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Gab_DialoGPT-medium_on_Gab_gedi_huggingface_1629485810_updated.json
Length of Hypothesis:  100


2021-08-21 19:03:27.134 | INFO     | __main__:metrics:39 - Key:Gab_Gab_politeness_updated_0--Politeness_Score :6



/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Gab_DialoGPT-medium_on_Gab_gedi_huggingface_1629489109_single.json
[['Emotion', 'joy']]
[1]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Gab_DialoGPT-medium_on_Gab_gedi_huggingface_1629489109_single.json
Length of Hypothesis:  100


2021-08-21 19:03:28.399 | INFO     | __main__:metrics:39 - Key:Gab_Gab_politeness_single_0--Politeness_Score :5



/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Gab_DialoGPT-medium_on_Gab_gedi_huggingface_1629484260_updated.json
[['Politeness', 'polite'], ['Emotion', 'joy']]
[10, 10]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Gab_DialoGPT-medium_on_Gab_gedi_huggingface_1629484260_updated.json
Length of Hypothesis:  100


...


/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Gab_DialoGPT-medium_on_Gab_gedi_huggingface_1629484920_updated.json
[['Politeness', 'polite']]
[30]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Gab_DialoGPT-medium_on_Gab_gedi_huggingface_1629484920_updated.json
Length of Hypothesis:  100


2021-08-21 19:03:34.431 | INFO     | __main__:metrics:39 - Key:Gab_Gab_politeness_updated_0--Politeness_Score :6



/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Gab_DialoGPT-medium_on_Gab_gedi_huggingface_1629550555_multi_2.json
[['Emotion', 'joy'], ['Politeness', 'polite']]
[1, 1]
/home/adarsh-binny/HULK_new/Counterspeech/Results/Generator-Gab_DialoGPT-medium_on_Gab_gedi_huggingface_1629550555_multi_2.json
Length of Hypothesis:  100


2021-08-21 19:03:35.321 | INFO     | __main__:metrics:39 - Key:Gab_Gab_politeness_2_0--Politeness_Score :6


TypeError: Object of type Tensor is not JSON serializable

****Toxicity_Score****

In [ ]:
def toxicity(trained_on,tested_on):
    train_path = glob.glob(path_datasets+'/*'+trained_on+'*/*rain*')[0]
    gen        = glob.glob(path_result+'/*'+trained_on+'*'+tested_on+'*1628*')
    ref        = glob.glob(path_result+'/*'+tested_on+'*'+'references'+'*')[0]
    
#     print(gen,ref)
    with open(ref, 'r') as file:
        ref_dict   = json.loads(file.read())
        
    scores = {}
    for files in gen:
        with open(files, 'r') as file:
            gen_dict  = json.loads(file.read())
        emotion   = gen_dict['params']['task_name'][0][1]
        gpu_id    = gen_dict['params']['gpu_id']
        
        detox = 0.0
        tot   = 0.0
        
        for key in gen_dict['samples']:
            for sentence in gen_dict['samples'][key]['counterspeech_model']:
                detox += get_non_toxicity_score(sentence)
                tot   += 1
        
        detox /= tot
        key = trained_on+'_'+tested_on+'_'+emotion+'_'+str(gpu_id)
        logger.info(f'Key -> {Key}-- Detox -> {detox}')

In [ ]:
toxicity('Reddit','Reddit')

In [ ]:
toxicity('Gab','Gab')

In [ ]:
toxicity('CONAN','CONAN')